# Set

In [5]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import warnings
import pandas as pd
import time

# 禁用警告
warnings.filterwarnings("ignore")
import requests

# 設定 User-Agent 和 Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# 發送請求，使用設定的 Headers
url = 'https://example.com'
response = requests.get(url, headers=headers)

# 使用 response 的內容進行後續處理
print(response.text)


<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

In [6]:
time.sleep(0.1)

# Grabs

In [7]:
# import requests
# from bs4 import BeautifulSoup
# import openpyxl

# # 定義基本URL和分頁區間
# base_url = "https://tos.fandom.com/zh/wiki/%E5%9C%96%E9%91%92/"
# page_ranges = [
# "1-300",
# # "301-600",
# # "601-900",
# # "901-1200",
# # "1201-1500",
# # "1501-1800",
# # "1801-2100",
# # "2101-2400",
# # "2401-2700",
# # "2701-3000",
# # "3001-3300",
# # "10001-10300",
# # "10301-10600",
# # "其它A-Z"
# ]
# # 定義自定義函數，判斷元素是否符合條件
# def has_custom_text(element):
#     if element is None:
#         return "None"
#     return element.has_attr("class") and "boxtest" in element["class"] and "隊伍技能" in element.get_text()

# # 創建一個Excel工作簿
# workbook = openpyxl.Workbook()
# # workbook2 = openpyxl.Workbook()
# print('creat excel')
# # for page_range in page_ranges:
# # sheet = workbook.create_sheet(title=f"Pages 1")
# print('start')
# # sub_sheet2 = workbook2.create_sheet(title=f"Page all")
# # sub_sheet2.append(["Image Name", "Data Url", "Data Text", "Data Text2"])

# for i in page_ranges:
#     page_url = f"{base_url}{i}"
#     print(page_url)
#     response = requests.get(page_url,  headers=headers)
#     print(response)

#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, "html.parser")
#         images = soup.find_all("a", href=True, title=True)
#         # time.sleep(0.1)
#         # print(images)
#         print('catch img')

#         # 在每個分頁中創建子工作表
#         sub_sheet = workbook.create_sheet(title=f"Page {i}")
#         sub_sheet.append(["Image Name", "Data Url", "Data Text", "Data Text2"]) # ["Image Name", "Table Data", "Table Text"]
        
#         for image in images:
#             image_name = image["title"]
#             images_replace = image["href"].replace("/zh/wiki/", "")
#             # print(image_name, images_replace)

#             card_url = f"https://tos.fandom.com/zh/wiki/{images_replace}"
#             # print(card_url)
#             card_response = requests.get(card_url,  headers=headers)
#             time.sleep(0.1)

#             if card_response.status_code == 200:
#                 card_soup = BeautifulSoup(card_response.content, "html.parser")
#                 time.sleep(0.1)
#                 # 找到所有包含指定樣式的元素
#                 # table = card_soup.find_all("table", {"id": "monster-data"}) # class="wikitable"   {"id": "monster-data"}
#                 # table = card_soup.select_one("table#monster-data") # class="wikitable"   {"id": "monster-data"}
#                 table = card_soup.select("table#monster-data") # class="wikitable"   {"id": "monster-data"}
#                 table_text = table[0].prettify() if len(table) > 0 else ""

#                 # elements = card_soup.find_all(has_custom_text)
#                 # time.sleep(0.1)

#                 if len(table) > 0:
#                     print(f'find {image_name}')
#                     # print(image_name, images_replace)
#                     # print(card_url)
#                     # 將表格數據添加到子工作表
#                     # table_text = str(table)
#                     elements = card_soup.find_all(has_custom_text)
#                     elements_text = str(elements)
#                     time.sleep(0.3)
#                     sub_sheet.append([image_name, card_url, table_text, elements_text])
#                     time.sleep(0.3)

#                     # sub_sheet2.append([image_name, card_url, table_text, elements_text])
#                     print(f'end {image_name}')
#                     del image_name, card_url, table_text, elements_text
#                     # workbook.save("monster_data_look.xlsx")
#         workbook.save("monster_data_look.xlsx")

#             # print(f"Processed card page {i}: {image_name}")

# # 刪除默認創建的工作表
# del workbook["Sheet"]
# # del workbook2["Sheet"]

# # 儲存Excel檔案
# workbook.save("monster_data.xlsx")
# # workbook2.save("monster_data_all_in_one.xlsx")
# print("Data saved to monster_data.xlsx")


In [8]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

# 定義基本URL和分頁區間
base_url = "https://tos.fandom.com/zh/wiki/%E5%9C%96%E9%91%92/"
page_ranges = [
"1-300",
"301-600",
"601-900",
"901-1200",
"1201-1500",
"1501-1800",
"1801-2100",
"2101-2400",
"2401-2700",
"2701-3000",
"3001-3300",
"10001-10300",
"10301-10600",
"其它A-Z"
]

# 定義自定義函數，判斷元素是否符合條件
def has_custom_text(element):
    if element is None:
        return "None"
    return element.has_attr("class") and "boxtest" in element["class"] and "隊伍技能" in element.get_text()

# 創建一個空的 DataFrame
data = pd.DataFrame(columns=["Image Name", "Data Url", "Data Text", "Data Text2"]) # ["Image Name", "Table Data", "Table Text"]

# 迴圈處理不同分頁範圍
for i in page_ranges:
    page_url = f"{base_url}{i}"
    print(page_url)
    response = requests.get(page_url)
    print(response)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        images = soup.find_all("a", href=True, title=True)
        print('catch img')

        for image in images:
            image_name = image["title"]
            images_replace = image["href"].replace("/zh/wiki/", "")
            card_url = f"https://tos.fandom.com/zh/wiki/{images_replace}"
            card_response = requests.get(card_url)
            time.sleep(0.1)

            if card_response.status_code == 200:
                card_soup = BeautifulSoup(card_response.content, "html.parser")
                time.sleep(0.1)
                table = card_soup.select("table#monster-data")  # class="wikitable"   {"id": "monster-data"}
                table_text = table[0].prettify() if len(table) > 0 else ""

                if len(table) > 0:
                    print(f'find {image_name}')
                    elements = card_soup.find_all(has_custom_text)
                    elements_text = str(elements)
                    # 將資料添加到 DataFrame
                    data = data.append({"Image Name": image_name,
                                        "Data Url": card_url,
                                        "Data Text": table_text,
                                        "Data Text2": elements_text}, ignore_index=True)
                    # data = data.append([image_name, card_url, table_text, elements_text])
                    print(f'end {image_name}')
                    data.to_excel("monster_data_look.xlsx", index=False)
                    del image_name, card_url, table_text, elements_text
                    time.sleep(0.1)


# 將 DataFrame 寫入 Excel 檔案
data.to_excel("monster_data.xlsx", index=False)
print("Data saved to monster_data.xlsx")


# # 116m 41.8s

# # 67m 39.5s
# # 78m 33.4s

# # 267m 44.5s


https://tos.fandom.com/zh/wiki/%E5%9C%96%E9%91%92/1-300
<Response [200]>
catch img
find No. 001 莫莉
end No. 001 莫莉
find No. 002 高級水術士莫莉
end No. 002 高級水術士莫莉
find No. 003 水魔法師莫莉
end No. 003 水魔法師莫莉
find No. 004 水元素賢者莫莉
end No. 004 水元素賢者莫莉
find No. 005 肖恩
end No. 005 肖恩
find No. 006 高級劍士肖恩
end No. 006 高級劍士肖恩
find No. 007 火焰劍士肖恩
end No. 007 火焰劍士肖恩
find No. 008 紅蓮劍士肖恩
end No. 008 紅蓮劍士肖恩
find No. 009 鄧肯
end No. 009 鄧肯
find No. 010 牧者鄧肯
end No. 010 牧者鄧肯
find No. 011 德魯依鄧肯
end No. 011 德魯依鄧肯
find No. 012 森林王者鄧肯
end No. 012 森林王者鄧肯
find No. 013 男爵奈寶尼
end No. 013 男爵奈寶尼
find No. 014 子爵奈寶尼
end No. 014 子爵奈寶尼
find No. 015 伯爵奈寶尼
end No. 015 伯爵奈寶尼
find No. 016 侯爵奈寶尼
end No. 016 侯爵奈寶尼
find No. 017 暗修導士安多
end No. 017 暗修導士安多
find No. 018 高級暗修導士安多
end No. 018 高級暗修導士安多
find No. 019 死靈術士安多
end No. 019 死靈術士安多
find No. 020 死靈祭師安多
end No. 020 死靈祭師安多
find No. 021 水方士
end No. 021 水方士
find No. 022 清川真人
end No. 022 清川真人
find No. 023 靈江仙人
end No. 023 靈江仙人
find No. 024 青龍孟章神君
end No. 024 青龍孟章神君
find No. 025 火方士
end No. 

In [9]:
list=[
"1-300",
"301-600",
"601-900",
"901-1200",
"1201-1500",
"1501-1800",
"1801-2100",
"2101-2400",
"2401-2700",
"2701-3000",
"3001-3300",
"10001-10300",
"10301-10600",
"其它A-Z"
]

In [10]:
# import pandas as pd

# # 讀取 Excel 檔案
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data.xlsx"
# xls = pd.ExcelFile(excel_file)

# # 建立一個新的 Excel 檔案
# new_excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\new_monster_data.xlsx"
# new_workbook = pd.ExcelWriter(new_excel_file, engine='xlsxwriter')

# # 遍歷每個分頁，重新命名列名並儲存到新的 Excel 檔案
# for sheet_name in xls.sheet_names:
#     df = pd.read_excel(xls, sheet_name)
    
#     # 確保讀取到的 df 是有效的 DataFrame
#     if not df.empty:
#         # 重新命名列名
#         new_columns = ["Image Name", "Data Url", "Data Text"] #, "Sheet Name"
#         df.columns = new_columns
#         # df["Sheet Name"] = sheet_name
#         # 將修改後的 DataFrame 寫入新的 Excel 檔案
#         df.to_excel(new_workbook, sheet_name, index=False)

# # 關閉新的 Excel 檔案
# new_workbook.save()
# new_workbook.close()

# print("Column names updated and saved to new Excel file:", new_excel_file)


# 合併sheets

In [11]:
import pandas as pd

# 讀取 Excel 檔案
excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data.xlsx"
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data_look.xlsx"
xls = pd.ExcelFile(excel_file)

# 建立一個新的 DataFrame 用來存儲合併後的數據
merged_data = pd.DataFrame(columns=["Image Name", "Data Url", "Data Text", "Data Text2", "Sheet Name"])

# 遍歷每個分頁並合併數據
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    
    # 新增 "Sheet Name" 欄位，填入對應的分頁名稱
    df["Sheet Name"] = sheet_name
    # df = pd.concat([df, sheet_name])
    # 將該分頁的數據合併到總的 DataFrame
    merged_data = pd.concat([merged_data, df])

# 建立一個新的 Excel 檔案
new_excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\merged_monster_data.xlsx"
merged_data.to_excel(new_excel_file, index=False)
print("Merged data saved to", new_excel_file)


Merged data saved to C:\\Users\\Administrator\\Desktop\\tower\\merged_monster_data.xlsx


In [12]:
# import pandas as pd
# from bs4 import BeautifulSoup

# # 讀取 Excel 檔案
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data_all_in_one.xlsx"
# df = pd.read_excel(excel_file)

# # 建立一個新的 DataFrame 用來存儲解析後的數據
# parsed_data_list = []

# # 遍歷每個資料列，進行 HTML 解析並提取對應的欄位
# for index, row in df.iterrows():
#     data_text = row["Data Text"]
#     soup = BeautifulSoup(data_text, "html.parser")
    
#     # 在這裡根據 HTML 的結構提取對應的欄位，以下只是一個示範
#     # 你需要根據實際的 HTML 結構進行解析
#     extracted_data = []
#     for element in soup.find_all("table", class_="wikitable"):
#         extracted_data.append(element.get_text())
    
#     # 將解析後的資料添加到新的 DataFrame
#     parsed_data_list.append({
#         "Image Name": row["Image Name"],
#         "Data Url": row["Data Url"],
#         "Data Text": data_text,
#         "Sheet Name": row["Sheet Name"],
#         "Parsed Data": extracted_data
#     })

# # 建立一個新的 DataFrame
# parsed_data = pd.DataFrame(parsed_data_list, columns=["Image Name", "Data Url", "Data Text", "Sheet Name", "Parsed Data"])

# # 建立一個新的 Excel 檔案
# new_excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\parsed_data.xlsx"
# parsed_data.to_excel(new_excel_file, index=False)
# print("Parsed data saved to", new_excel_file)


# 修正文字資料格式

In [13]:
import pandas as pd
from bs4 import BeautifulSoup

# 讀取 Excel 檔案
excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\merged_monster_data.xlsx"
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data_look.xlsx"
parsed_data = pd.read_excel(excel_file)

# 建立一個空的 DataFrame 用來存儲新的 Excel 檔案資料
new_data = pd.DataFrame()
aa = 0
# 遍歷每一行資料
for index, row in parsed_data.iterrows():
    # 解析 "Data Text" 欄位的 HTML 內容
    soup_text = BeautifulSoup(row["Data Text"], "html.parser")
    aa+=1
    print(aa)

    image_link = soup_text.select('a', class_='image')[0]['href'].replace('amp;','')
    if aa == 1:
        print(image_link)
    if aa%10 == 0:
        print(image_link)
    # 使用 CSS 選擇器定位屬性和種族的元素
    attributes_element = soup_text.select_one("th:contains('屬性') + td")
    race_element = soup_text.select_one("th:contains('種族') + td")
    # 提取所需資料
    # 提取屬性和種族資訊
    attributes = attributes_element.get_text(strip=True) if attributes_element else "error"
    race = race_element.get_text(strip=True) if race_element else "error"

    # 使用 CSS 選擇器定位帶有特定文字的 <span> 元素
    min_cd_span = soup_text.select_one("span:contains('最小')")
    # 獲取 <td> 元素的內容
    try:
        min_cd_content = min_cd_span.find_next("td").get_text(strip=True)
        min_cd = "最小"+ min_cd_span.get_text(strip=True).split("最小")[1] 
    except:
        min_cd_content = "無"
        min_cd = "最小CD"

    # 使用 CSS 選擇器定位主動技名稱和主動技效果的元素
    active_skill_name_elements = soup_text.select("th + td span.ability-tooltip a")
    # active_skill_effect_element = soup_text.select_one("th:contains('效果') + td")
    active_skill_effect_elements = soup_text.select("th:contains('效果') + td")
    # 
    # 判斷主動技名稱個數，如果大於2，則處理為多個主動技名稱，否則為一個主動技名稱
    if len(active_skill_name_elements) > 2:
        active_skill_names = [element.get_text(strip=True).replace(" ", "") for element in active_skill_name_elements[:-1]]  # 排除最後一個元素
    else:
        active_skill_name = active_skill_name_elements[0].get_text(strip=True).replace(" ", "") if active_skill_name_elements else "未找到主動技名稱"

    # 判斷主動技效果個數，如果大於2，則處理為多個主動技效果，否則為一個主動技效果
    if len(active_skill_effect_elements) > 2:
        active_skill_effects = [element.get_text(strip=True).replace(" ", "") for element in active_skill_effect_elements[:-1]]  # 排除最後一個元素
    else:
        active_skill_effect = active_skill_effect_elements[0].get_text(strip=True).replace(" ", "") if active_skill_effect_elements else "未找到主動技效果"

    # 使用 CSS 選擇器定位隊長技名稱和隊長技效果的元素
    leader_skill_name_element = soup_text.select_one("td:contains('隊長技') + th + td span.ability-tooltip a")
    leader_skill_effect_element = soup_text.select("th:contains('效果') + td")

    try:
        # 提取隊長技名稱和隊長技效果
        leader_skill_name = leader_skill_name_element.get_text(strip=True).replace(" ", "") if leader_skill_name_element else "未找到隊長技名稱"
        leader_skill_effect = leader_skill_effect_element[-1].get_text(strip=True).replace(" ", "") if leader_skill_effect_element else "未找到隊長技效果"
    except:
        leader_skill_name = "無"    
        leader_skill_effect = "無"

    data_text = {
        "圖片": image_link,
        "屬性": attributes,
        "種族": race,
        min_cd: min_cd_content,
        "主動技名稱": active_skill_names if len(active_skill_name_elements) > 2 else active_skill_name,
        "主動技效果": active_skill_effects if len(active_skill_effect_elements) > 2 else active_skill_effect,
        "隊長技名稱": leader_skill_name,
        "隊長技效果": leader_skill_effect
    }

    # 解析 "Data Text2" 欄位的 HTML 內容
    soup_text2 = BeautifulSoup(row["Data Text2"], "html.parser")
    # 找到包含 "隊伍技能" 文字的 <div> 元素，提取隊伍技能的文字內容
    try:
        data_text2 = soup_text2.select_one("div:contains('隊伍技能')").get_text(strip=True) 
    except:
        data_text2 = "無隊伍技能"
    # data_text2 = soup_text2.get_text()

    # 將未處理的其他欄位資料也添加到新的資料字典中 # "Image Name", "Data Url", "Data Text", "Data Text2"
    new_data_row = {
        "Image Name": row["Image Name"],
        "Data Url": row["Data Url"],
        **data_text,
        # "Data Text": data_text,
        "Team Skills": data_text2
    }

    # 將整行資料添加到新的 DataFrame 中
    new_data = new_data.append(new_data_row, ignore_index=True)

# 將新的資料寫入新的 Excel 檔案
new_excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\processed_monster_data.xlsx"
new_data.to_excel(new_excel_file, index=False)
print("Processed data saved to", new_excel_file)

# 4108


1
https://static.wikia.nocookie.net/tos/images/2/2f/001i.png/revision/latest?cb=20140709005649&path-prefix=zh
2
3
4
5
6
7
8
9
10
https://static.wikia.nocookie.net/tos/images/c/c9/010i.png/revision/latest?cb=20140618120557&path-prefix=zh
11
12
13
14
15
16
17
18
19
20
https://static.wikia.nocookie.net/tos/images/9/95/020i.png/revision/latest?cb=20140618122126&path-prefix=zh
21
22
23
24
25
26
27
28
29
30
https://static.wikia.nocookie.net/tos/images/1/14/030i.png/revision/latest?cb=20140618124041&path-prefix=zh
31
32
33
34
35
36
37
38
39
40
https://static.wikia.nocookie.net/tos/images/f/f9/040i.png/revision/latest?cb=20140618130223&path-prefix=zh
41
42
43
44
45
46
47
48
49
50
https://static.wikia.nocookie.net/tos/images/6/67/050i.png/revision/latest?cb=20140618131432&path-prefix=zh
51
52
53
54
55
56
57
58
59
60
https://static.wikia.nocookie.net/tos/images/4/48/060i.png/revision/latest?cb=20140618132642&path-prefix=zh
61
62
63
64
65
66
67
68
69
70
https://static.wikia.nocookie.net/tos/image

## test

In [14]:
# 讀取 Excel 檔案
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\merged_monster_data.xlsx"
excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\merged_monster_data.xlsx"
parsed_data = pd.read_excel(excel_file)

# # 建立一個空的 DataFrame 用來存儲新的 Excel 檔案資料
# new_data = pd.DataFrame()

In [15]:
# c2=0
# for i in range (0,4080):
#     soup_text = BeautifulSoup(parsed_data.iloc[i]["Data Text"], "html.parser")
#     if len(soup_text)==2:
#         c2+=1

# print(c2)

In [16]:
KK = 8
soup_text = BeautifulSoup(parsed_data.iloc[KK]["Data Text"], "html.parser")
soup_text

<table class="wikitable" id="monster-data" style="width: 100%; text-align:center; font-size:9.5pt; margin: 1em auto 1em auto;">
<tbody>
<tr>
<td class="attr-e" rowspan="3" style="width:15%; vertical-align:center;">
<div class="hidden">
<span class="imgsrc-tooltip" data-imgsrc="009" style="cursor:pointer; width:520px;">
<a class="image" href="https://static.wikia.nocookie.net/tos/images/a/a2/009i.png/revision/latest?cb=20140618120355&amp;path-prefix=zh">
<img alt="009i" class="lazyload" data-image-key="009i.png" data-image-name="009i.png" data-relevant="1" data-src="https://static.wikia.nocookie.net/tos/images/a/a2/009i.png/revision/latest/scale-to-width-down/100?cb=20140618120355&amp;path-prefix=zh" decoding="async" height="100" loading="lazy" src="data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D" width="100"/>
</a>
</span>
</div>
</td>
<th style="width:8.5%; height:50px;">
    名稱
   </th>
<td colspan="5" style="font-size:1.7em">
<b>
     鄧肯
    </

In [17]:
from PIL import Image
import requests
from io import BytesIO

image_link = soup_text.select('a', class_='image')[0]['href'].replace('amp;','')
# 获取图像内容
response = requests.get(image_link)
image_data = BytesIO(response.content)
print(image_link)
# 打开图像并显示
image = Image.open(image_data)
image.show()

https://static.wikia.nocookie.net/tos/images/a/a2/009i.png/revision/latest?cb=20140618120355&path-prefix=zh


In [18]:
attributes_element = soup_text.select_one("th:contains('屬性') + td")
race_element = soup_text.select_one("th:contains('種族') + td")
# 提取所需資料
# 提取屬性和種族資訊
attributes = attributes_element.get_text(strip=True) if attributes_element else "error"
race = race_element.get_text(strip=True) if race_element else "error"

print(attributes, race)

木 人類


In [19]:
# 使用 CSS 選擇器定位帶有特定文字的 <span> 元素
min_cd_span = soup_text.select_one("span:contains('最小')")
# 獲取 <td> 元素的內容
try:
    min_cd_content = min_cd_span.find_next("td").get_text(strip=True)
    min_cd = "最小"+ min_cd_span.get_text(strip=True).split("最小")[1] 
except:
    min_cd_content = "無"
    min_cd = "技能CD"
print(min_cd, min_cd_content)

最小CD 5


In [20]:
# 使用 CSS 選擇器定位主動技名稱和主動技效果的元素
active_skill_name_elements = soup_text.select("th + td span.ability-tooltip a")
# active_skill_effect_element = soup_text.select_one("th:contains('效果') + td")
active_skill_effect_elements = soup_text.select("th:contains('效果') + td")
# 
# 判斷主動技名稱個數，如果大於2，則處理為多個主動技名稱，否則為一個主動技名稱
if len(active_skill_name_elements) > 2:
    active_skill_names = [element.get_text(strip=True) for element in active_skill_name_elements[:-1]]  # 排除最後一個元素
else:
    active_skill_name = active_skill_name_elements[0].get_text(strip=True) if active_skill_name_elements else "未找到主動技名稱"

# 判斷主動技效果個數，如果大於2，則處理為多個主動技效果，否則為一個主動技效果
if len(active_skill_effect_elements) > 2:
    active_skill_effects = [element.get_text(strip=True) for element in active_skill_effect_elements[:-1]]  # 排除最後一個元素
else:
    active_skill_effect = active_skill_effect_elements[0].get_text(strip=True) if active_skill_effect_elements else "未找到主動技效果"

# 使用 CSS 選擇器定位隊長技名稱和隊長技效果的元素
leader_skill_name_element = soup_text.select_one("td:contains('隊長技') + th + td span.ability-tooltip a")
leader_skill_effect_element = soup_text.select("th:contains('效果') + td")

try:
    # 提取隊長技名稱和隊長技效果
    leader_skill_name = leader_skill_name_element.get_text(strip=True) if leader_skill_name_element else "未找到隊長技名稱"
    leader_skill_effect = leader_skill_effect_element[-1].get_text(strip=True) if leader_skill_effect_element else "未找到隊長技效果"
except:
    leader_skill_name = "無"    
    leader_skill_effect = "無"

print("主動技名稱:", active_skill_names if len(active_skill_name_elements) > 2 else active_skill_name)
print("主動技效果:", active_skill_effects if len(active_skill_effect_elements) > 2 else active_skill_effect)
print("隊長技名稱:", leader_skill_name)
print("隊長技效果:", leader_skill_effect)


主動技名稱: 符石解放 ‧ 木
主動技效果: 引爆火符石造成敵全體木屬性傷害
隊長技名稱: 森之力
隊長技效果: 木屬性攻擊力 1.5 倍


In [21]:
# 解析 "Data Text2" 欄位的 HTML 內容
soup_text2 = BeautifulSoup(parsed_data.iloc[KK]["Data Text2"], "html.parser")
# 找到包含 "隊伍技能" 文字的 <div> 元素，提取隊伍技能的文字內容
try:
    data_text2 = soup_text2.select_one("div:contains('隊伍技能')").get_text(strip=True) 
except:
    data_text2 = "無隊伍技能"
print(data_text2)


隊伍技能隊伍技能：◆ 「主角」系列的回復力基值提升⇒ 最大 1.5 倍發動條件：以 ≥3 個不同屬性、相同星數的「主角」系列作成員


In [22]:
# import pandas as pd
# from bs4 import BeautifulSoup

# # 讀取 Excel 檔案
# excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\monster_data_all_in_one.xlsx"
# parsed_data = pd.read_excel(excel_file)

# # 建立一個空的 DataFrame 用來存儲新的 Excel 檔案資料
# new_data = pd.DataFrame()

# # 遍歷每一行資料
# for index, row in parsed_data.iterrows():
#     # 解析 "Data Text" 欄位的 HTML 內容
#     soup_text = BeautifulSoup(row["Data Text"], "html.parser")

#     # 提取所需資料
#     attributes = soup_text.find("td", string="屬性").find_next("td").get_text(strip=True)
#     race = soup_text.find("td", string="種族").find_next("td").get_text(strip=True)

#     # 找到包含 "最小CD" 文字的 <span> 元素
#     min_cd = soup_text.find("span", text="最小")
#     # 獲取 <td> 元素的內容
#     min_cd_content = min_cd.find_next("td").get_text(strip=True)
#     min_cd = min_cd.get_text(strip=True)

#     active_skill_name = soup_text.find("td", string="主動技").find_next("span", class_="ability-tooltip").a.get_text(strip=True)
#     active_skill_effect = soup_text.find("td", string="主動技效果").find_next("td").get_text(strip=True)
#     leader_skill_name = soup_text.find("td", string="隊長技").find_next("span", class_="ability-tooltip").a.get_text(strip=True)
#     leader_skill_effect = soup_text.find("td", string="隊長技效果").find_next("td").get_text(strip=True)

#     # 提取 "Data Text" 中的資料項目並組成字典
#     data_text_items = {
#         "屬性": attributes,
#         "種族": race,
#         min_cd: min_cd_content,
#         "主動技名稱": active_skill_name,
#         "主動技效果": active_skill_effect,
#         "隊長技名稱": leader_skill_name,
#         "隊長技效果": leader_skill_effect
#     }

#     # 解析 "Data Text2" 欄位的 HTML 內容
#     soup_text2 = BeautifulSoup(row["Data Text2"], "html.parser")
#     # 找到包含 "隊伍技能" 文字的 <div> 元素，提取隊伍技能的文字內容
#     data_text2 = soup_text2.find("div", string="隊伍技能").get_text(strip=True)

#     # data_text2 = soup_text2.get_text()

#     # 將未處理的其他欄位資料也添加到新的資料字典中 # "Image Name", "Data Url", "Data Text", "Data Text2"
#     new_data_row = {
#         "Image Name": row["Image Name"],
#         "Data Url": row["Data Url"],
#         **data_text_items,
#         "隊伍技能": data_text2
#     }

#     # 將整行資料添加到新的 DataFrame 中
#     new_data = new_data.append(new_data_row, ignore_index=True)

# # 將新的資料寫入新的 Excel 檔案
# new_excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\new_processed_data.xlsx"
# new_data.to_excel(new_excel_file, index=False)
# print("Processed data saved to", new_excel_file)


# Json

In [23]:
import pandas as pd

# 讀取 Excel 檔案
excel_file = r"C:\\Users\\Administrator\\Desktop\\tower\\processed_monster_data.xlsx"
parsed_data = pd.read_excel(excel_file)

# 將 DataFrame 轉換為 JSON 格式，並保留中文原始文字
json_file = r"C:\\Users\\Administrator\\Desktop\\tower\\final_data.json"
parsed_data.to_json(json_file, orient="records", indent=4, force_ascii=False)

print("Data saved as JSON to", json_file)


Data saved as JSON to C:\\Users\\Administrator\\Desktop\\tower\\final_data.json


# Add New Cards

In [24]:
# import pandas as pd

# # 讀取兩個 Excel 檔案
# excel_file1 = r"C:\\Users\\Administrator\\Desktop\\tower\\processed_monster_data.xlsx"
# excel_file2 = r"C:\\Users\\Administrator\\Desktop\\tower\\processed_monster_data.xlsx"


# # 讀取 Excel 檔案到 DataFrame
# df1 = pd.read_excel(excel_file1, index_col=0)  # 使用第一個檔案的索引作為索引列
# df2 = pd.read_excel(excel_file2)

# # 合併兩個 DataFrame，忽略 df2 的索引
# combined_df = pd.concat([df1, df2], ignore_index=True)

# # 將合併後的 DataFrame 存為新的 Excel 檔案
# output_file = "combined_file.xlsx"
# combined_df.to_excel(output_file, index=True)  # 儲存索引

# print("合併完成，結果已儲存為 combined_file.xlsx")


# Creat Json_Html

In [26]:
import json

# 读取原始的JSON数据
with open('final_data.json', 'r', encoding='utf-8') as json_file:
    original_data = json.load(json_file)

# 处理数据并生成HTML格式
html_data = []
for item in original_data:
    html_item = {
        "Image Name": item['Image Name'],
        "HTML": f"""
        <div class="card">
            <h3>{item['Image Name']}</h3>
            <p>屬性: {item['屬性']}</p>
            <p>種族: {item['種族']}</p>
            <!-- 其他字段以类似的方式添加 -->
        </div>
        """
    }
    html_data.append(html_item)

# 将处理后的HTML数据保存为新的JSON文件
with open('processed_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(html_data, json_file, ensure_ascii=False, indent=4)


In [ ]:
<table class="table"><tr class="table-row"><th class="table-cell" style="width: 140px; max-height: 32px; overflow-y: hidden;">名稱</th><th class="table-cell" style="width: 50px; max-height: 32px; overflow-y: hidden;">詳細介紹</th><th class="table-cell" style="width: 50px; max-height: 32px; overflow-y: hidden;">屬性</th><th class="table-cell" style="width: 50px; max-height: 32px; overflow-y: hidden;">種族</th><th class="table-cell" style="width: 50px; max-height: 32px; overflow-y: hidden;">最小CD</th><th class="table-cell" style="width: 140px; max-height: 32px; overflow-y: hidden;">主動技名稱</th><th class="table-cell" style="width: 175px; max-height: 32px; overflow-y: hidden;">主動技效果</th><th class="table-cell" style="width: 140px; max-height: 32px; overflow-y: hidden;">隊長技名稱</th><th class="table-cell" style="width: 175px; max-height: 32px; overflow-y: hidden;">隊長技效果</th><th class="table-cell" style="width: 175px; max-height: 32px; overflow-y: hidden;">隊伍技能</th><th class="table-cell" style="width: 50px; max-height: 32px; overflow-y: hidden;">最小EP</th></tr><tr class="table-row"><td class="table-cell" style="width: 140px; overflow-y: auto;">No. 001 莫莉</td><td class="table-cell" style="width: 50px; overflow-y: auto;"><a href="https://tos.fandom.com/zh/wiki/%E8%8E%AB%E8%8E%89" target="_blank">點擊前往</a></td><td class="table-cell" style="width: 50px; overflow-y: auto;">https://static.wikia.nocookie.net/tos/images/2/2f/001i.png/revision/latest?cb=20140709005649&amp;path-prefix=zh</td><td class="table-cell" style="width: 50px; overflow-y: auto;">水</td><td class="table-cell" style="width: 50px; overflow-y: auto;">人類</td><td class="table-cell" style="width: 140px; overflow-y: auto;">5</td><td class="table-cell" style="width: 175px; overflow-y: auto;">符石解放‧水</td><td class="table-cell" style="width: 140px; overflow-y: auto;">引爆木符石造成敵全體水屬性傷害</td><td class="table-cell" style="width: 175px; overflow-y: auto;">海之力</td><td class="table-cell" style="width: 175px; overflow-y: auto;">水屬性攻擊力1.5倍</td><td class="table-cell" style="width: 50px; overflow-y: auto;">隊伍技能隊伍技能：◆ 「主角」系列的回復力基值提升⇒ 最大 1.5 倍發動條件：以 ≥3 個不同屬性、相同星數的「主角」系列作成員</td></tr></table>